#  Interactive Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

* TASK 1: Add a Launch Site Drop-down Input Component
* TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
* TASK 3: Add a Range Slider to Select Payload
* TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

* Which site has the largest successful launches?
* Which site has the highest launch success rate?
* Which payload range(s) has the highest launch success rate?
* Which payload range(s) has the lowest launch success rate?
* Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
  launch success rate?

In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
# MY IMPORTS
import dash_table

# Read the airline data into pandas dataframe 
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

##  <MY_ADDS>
dropdown_list = spacex_df[['Launch Site']].drop_duplicates(inplace=False)
df2 = pd.DataFrame({'Launch Site':['ALL']})
dropdown_list = dropdown_list.append(df2)
dropdown_list = dropdown_list.sort_values(['Launch Site']).reset_index(drop=True)

#  Alternative method provided by instructor
if 1==0:
    OptionList = [{'label': i, 'value': i} for i in spacex_df['Launch Site'].unique()]
    OptionList.insert(0,{'label': 'All', 'value': 'All'})

# Default Pie Chart
fig_default_pie = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches by Site')
fig_default_scatter = px.scatter(spacex_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')

##  </MY_ADDS>


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    dcc.Dropdown(
                                        id='site_dropdown'
                                        , options=[
                                            {'label': i, 'value': i} for i in dropdown_list['Launch Site']
                                        ]
                                        # alternative using method provided by instructor
                                        #, options=OptionList
                                        #, value=OptionList[0]
                                        , value='ALL'
                                        , placeholder="Select a Launch Site here"
                                        , searchable=True
                                        , style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}
                                    )
                                ])

                                , html.Br()

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                , html.Div(
                                    #dcc.Graph(id='success-pie-chart')
                                    # default it to all for now so we can confirm it displays
                                    dcc.Graph(id='success-pie-chart', figure=fig_default_pie)
                                )
                                , html.Br()
                                , html.Div(['Initial View - ALL'], id='callback_test')
                                , html.Br()

                                , html.P("Payload range (Kg):")
                                # TASK 3: Add a slider to select payload range
                                , dcc.RangeSlider(
                                    id='payload-slider'
                                    , min=0
                                    , max=10000
                                    , step=100
                                    , value=[min_payload, max_payload]
                                    , marks={
                                        0: '0'
                                        , 2500: '2500'
                                        , 5000: '5000'
                                        , 7500: '7500'
                                        , 10000: '10000'
                                    }
                                )

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                , html.Div(
                                    dcc.Graph(
                                        id='success-payload-scatter-chart'
                                        , figure=fig_default_scatter
                                    )
                                )
                                # TEST - TASK 4
                                , html.Br()
                                # , html.Div(
                                #     id='task4_test'
                                #     , title='TASK 4 TEST - Show Dataframe'
                                #     , children = ['Initial View - ALL']
                                # )
                                # , html.Br()
                                # ,
                            ])  # END - Main app.layout Div
        

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure')
    , Input(component_id='site_dropdown', component_property='value')
)
def get_pie_chart(site_val):
    filtered_df = spacex_df
    if site_val == 'ALL':
        fig = px.pie(
            filtered_df
            , values='class'
            , names= 'Launch Site'
            , title = 'Total Success Launches By Site'
        )
        return fig
        #return[dcc.Graph(figure=fig)]
    else:
        # return the outcomes pichart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==site_val]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index()
        filtered_df.rename(columns={0:'class count'}, inplace=True)
        fig = px.pie(
            filtered_df
            , values='class count'
            , names= 'class'
            , title = f'Total Success Launches for Site {site_val}'
        )    
        return fig 


##  MY TEST of dropdown callback
@app.callback(
    Output(component_id='callback_test', component_property='children')
    , Input(component_id='site_dropdown', component_property='value')
)
def callbacktest(siteval):
    retval = f'siteval = {siteval}'
    return retval


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure')
    , [
        Input(component_id='site_dropdown', component_property='value')
        , Input(component_id='payload-slider', component_property='value')
    ]
)
def get_scatter_plot(site_val, slider_val):
    low, high = slider_val
    cond = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    filtered_df = spacex_df[cond]
    if site_val == 'ALL':
        fig = px.scatter(
            filtered_df
            , x='Payload Mass (kg)'
            , y='class'
            , color='Booster Version Category'
        )
        return fig
    else:
        # return the outcomes pichart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==site_val]
        filtered_df = filtered_df.groupby(['Launch Site', 'Payload Mass (kg)', 'Booster Version Category','class']).size().reset_index()
        filtered_df.rename(columns={0:'class count'}, inplace=True)
        fig = px.scatter(
            filtered_df
            , x='Payload Mass (kg)'
            , y='class'
            , color='Booster Version Category'
        )
        return fig 

# Run the app
if __name__ == '__main__':
    app.run_server()



<ipython-input-2-530868fb4f61>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-530868fb4f61>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-530868fb4f61>:8: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
